In [47]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import sys
import glob
import pickle
import generate_palette as gp

In [2]:
with open('dataset/clt16.pckl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    centers = u.load()

In [55]:
def load_image(path, centers):
    """Given the list of data to be used, return a list of a nest list of
    16 percentages for the percentages of pixels that is the closest to 
    each cluster center
    
    Returns: a 2D list of percentage
    
    Arguments:
        paths (list): a list of path to the datasets
        centroids (list): a list of cluster centers
    """
    def cluster_around(flattened_img, centroids):
        """Given a reshaped 2d numpy arrary of size n*3, this function
        loops through each element in the 2d numpy array and each cluster
        center, finds which cluster center is closest to that element, and 
        add one to the value of the element in a dictionary with the cluster 
        center as its key  

        Returns: a normalized dictionary with each key being a cluster center

        Arguments:
            paths (list): a list of path to the datasets
            centroids (list): a list of cluster centers
        """
        cluster = dict((tuple(centroid), 0) for centroid in centroids)
        total = float(len(flattened_img))
        for i in flattened_img:
            temp = 0
            first_pass = True
            which_centroid = None
            for centroid in centroids:
                dist = np.linalg.norm((i-centroid))
                if first_pass:
                    which_centroid = centroid
                    temp = dist
                    first_pass = False
                elif temp > dist:
                    which_centroid = centroid
                    temp = dist
            cluster[tuple(which_centroid)] += 1
        cluster = dict((k, v/total) for k, v in cluster.items())
        return cluster
    
    # Read images with pillow and turn then into more general purpose
    # numpy arrays
    image = np.array(Image.open(path))

    # resize the image to a lower dimension for less computing time
    image = gp.edit_image(image)
    cluster = cluster_around(image, centers)
    cluster = [v for _, v in cluster.items()]
    
    return cluster

In [30]:
with tf.Graph().as_default():
    # Inject placeholder into the graph
    
    cluster = load_image(path)
    output = tf.placeholder(tf.float32, shape=(4, ))
    
    
    export_dir = 'dataset/dense-export/2'
        
    with tf.Session() as sess:
        # Restore the model from last checkpoints
    
        loader = tf.train.import_meta_graph('dataset/model_sess/model.ckpt-30000.meta')
        loader.restore(sess, 'dataset/model_sess/model.ckpt-30000')

        builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

        predict_tensor_inputs_info = tf.saved_model.utils.build_tensor_info(cluster)
        predict_tensor_scores_info = tf.saved_model.utils.build_tensor_info(output)

        prediction_signature = (
                    tf.saved_model.signature_def_utils.build_signature_def(
                        inputs={'cluster': predict_tensor_inputs_info},
                        outputs={'scores': predict_tensor_scores_info},
                        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

        legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

        builder.add_meta_graph_and_variables(
                    sess, [tf.saved_model.tag_constants.SERVING],
                    signature_def_map={
                        'predict_cluster': prediction_signature
                    },
                    legacy_init_op=legacy_init_op)

        builder.save()

NameError: name 'path' is not defined

In [31]:
tfimg = tf.image.decode_jpeg('image/image_3.jpg', 3)

In [38]:
image = tf.image.convert_image_dtype(tfimg, dtype=tf.float32)

In [39]:
image

<tf.Tensor 'convert_image:0' shape=(?, ?, 3) dtype=float32>

In [40]:
image = tf.expand_dims(image, 0)

In [41]:
image

<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?, 3) dtype=float32>

In [42]:
image = tf.subtract(image, 0.5)
image = tf.multiply(image, 2.0)

In [74]:
a = load_image('image/image_3.jpg',centers)

In [75]:
a

[0.006486486486486486,
 0.6032432432432432,
 0.0,
 0.04418918918918919,
 0.004324324324324324,
 0.0,
 0.0477027027027027,
 0.0,
 0.07864864864864865,
 0.0,
 0.0002702702702702703,
 0.0035135135135135136,
 0.0013513513513513514,
 0.20810810810810812,
 0.0,
 0.002162162162162162]

In [76]:
f = open('image/image_3.pckl', 'wb')
pickle.dump(a, f)
f.close()

In [63]:
a = tf.convert_to_tensor(a, dtype=tf.float32)

In [66]:
tf.constant(a)

TypeError: List of Tensors when single Tensor expected